In [7]:
import os
from dotenv import load_dotenv
import pinecone # Pinecone 사용을 위해 추가
from pathlib import Path
import json

# pip install -r requirements.txt

# .env 파일 로드
load_dotenv()

# OpenAI API 키 (환경변수에서 로드)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# --- Pinecone 설정 추가 ---
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
INDEX_NAME = os.getenv("INDEX_NAME", "ddauction-db")
# -------------------------

# 현재 코드에서 사용되지 않는 변수들은 그대로 유지
IMAGE_DIR = os.getenv('IMAGE_DIR', 'image_jpg')

# API 키 및 Pinecone 설정 검증
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다. .env 파일 또는 환경변수를 확인하세요.")

if not PINECONE_API_KEY or not PINECONE_ENVIRONMENT:
    print("⚠️  PINECONE_API_KEY 또는 PINECONE_ENVIRONMENT가 설정되지 않았습니다. .env 파일을 확인하세요.")


In [8]:
import fitz  # PyMuPDF
import os
from pathlib import Path
from typing import Dict, Union

# =========================================================
# 1. 파일 경로 및 문서 매핑 설정
# =========================================================

# PDF 파일이 저장된 디렉토리 (현재 스크립트가 실행되는 곳의 'pdfs' 폴더)
PDF_DIR = Path("./pdfs")

# 변환된 Markdown 파일을 저장할 디렉토리 (LangGraph API 폴더)
MD_DIR = Path("./rag-api/docs")
MD_DIR.mkdir(parents=True, exist_ok=True)

# 🚨 중요: PDF 원본 파일 이름(값)과 RAG 시스템의 고정 식별자(키) 매핑
# RAG 시스템의 분류 로직은 이 키(MD 파일 이름, 확장자 제외)를 사용합니다.
# 이제 MD 파일 이름과 PDF 파일 이름이 확장자만 다르고 동일합니다.
PDF_MAPPING: Dict[str, str] = {
    # [RAG 식별자 (MD 파일 이름)] : [PDF 원본 파일 이름]
    "auction_policy.md": "auction_policy.pdf",
    "used_item_guidelines.md": "used_item_guidelines.pdf",
    "general_service_policy.md": "general_service_policy.pdf",
    "shipping_delivery_policy.md": "shipping_delivery_policy.pdf",
    "prohibited_items_policy.md": "prohibited_items_policy.pdf",
    "sales_and_returns_faq.md": "sales_and_returns_faq.pdf",
}

# =========================================================
# 2. PDF to MD 변환 함수
# =========================================================

def pdf_to_md(pdf_path: Path, md_path: Path):
    """
    단일 PDF 파일을 읽어 Markdown 파일로 저장합니다.
    """
    if not pdf_path.exists():
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path.resolve()}")
        return

    doc: Union[fitz.Document, None] = None

    try:
        doc = fitz.open(pdf_path)

        print(f"--- '{pdf_path.name}' 변환 시작 ---")

        with open(md_path, "w", encoding="utf-8") as f:
            for page in doc:
                text = page.get_text("text")
                # 페이지 구분을 위한 Markdown 표시
                f.write(f"\n\n## Page {page.number + 1}\n")
                f.write(text.strip() + "\n")

        print(f"✅ Markdown 파일 생성 완료: {md_path.name} -> {md_path.resolve()}")

    except Exception as e:
        print(f"❌ 변환 중 오류 발생 (파일: {pdf_path.name}): {e}")
    finally:
        if doc:
            doc.close()


# =========================================================
# 3. 메인 실행 (일괄 변환)
# =========================================================

if __name__ == "__main__":

    print("--- RAG 문서 일괄 PDF to Markdown 변환 시작 ---")

    # PDF_MAPPING을 순회하며 모든 파일 변환
    for md_filename, pdf_filename in PDF_MAPPING.items():
        pdf_path = PDF_DIR / pdf_filename
        md_path = MD_DIR / md_filename

        pdf_to_md(pdf_path, md_path)

    print("\n--- 일괄 변환 종료 ---")
    print("📢 **필수 후속 조치:**")
    print("1. 'pdfs' 폴더에 위 리스트와 정확히 일치하는 PDF 파일 6개를 넣어주세요.")
    print("2. 'rag-api' 폴더에 생성된 모든 .md 파일을 열어 내용을 수동으로 정리해야 합니다.")

--- RAG 문서 일괄 PDF to Markdown 변환 시작 ---
--- 'auction_policy.pdf' 변환 시작 ---
✅ Markdown 파일 생성 완료: auction_policy.md -> C:\mirae\2차프로젝트\ddauction-project\chatbot\rag-api\docs\auction_policy.md
❌ PDF 파일을 찾을 수 없습니다: C:\mirae\2차프로젝트\ddauction-project\chatbot\pdfs\used_item_guidelines.pdf
❌ PDF 파일을 찾을 수 없습니다: C:\mirae\2차프로젝트\ddauction-project\chatbot\pdfs\general_service_policy.pdf
❌ PDF 파일을 찾을 수 없습니다: C:\mirae\2차프로젝트\ddauction-project\chatbot\pdfs\shipping_delivery_policy.pdf
❌ PDF 파일을 찾을 수 없습니다: C:\mirae\2차프로젝트\ddauction-project\chatbot\pdfs\prohibited_items_policy.pdf
❌ PDF 파일을 찾을 수 없습니다: C:\mirae\2차프로젝트\ddauction-project\chatbot\pdfs\sales_and_returns_faq.pdf

--- 일괄 변환 종료 ---
📢 **필수 후속 조치:**
1. 'pdfs' 폴더에 위 리스트와 정확히 일치하는 PDF 파일 6개를 넣어주세요.
2. 'rag-api' 폴더에 생성된 모든 .md 파일을 열어 내용을 수동으로 정리해야 합니다.
